# Preprocessing: BJH016


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
# matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [ ]:
%matplotlib qt5

In [2]:
## Prep paths ##

subject = 'BJH016'
orig_data_fi = '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH016/PacmanTask/ECOGS001R01.dat'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/BJH016/ieeg/BJH016_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 1963999 =      0.000 ...   982.000 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH016/ieeg/BJH016_notched_filtered_clean_ieeg-1.fif...
    Range : 1964000 ... 3927999 =    982.000 ...  1963.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH016/ieeg/BJH016_notched_filtered_clean_ieeg-2.fif...
    Range : 3928000 ... 5891999 =   1964.000 ...  2945.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH016/ieeg/BJH016_notched_filtered_clean_ieeg-3.fif...
    Range : 5892000 ... 5901899 =   2946.000 ...  2950.950 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH016/ieeg/BJH016_raw_clean_ieeg.fif...
    Range : 0 ... 1963999 =      0.000 ...   982.000 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH016/ieeg/BJH016_raw_clean_ieeg-1.fif...
    Range : 1964000 ... 3927999 =    982.000 

In [4]:
raw_clean_fif.info

<Info | 11 non-empty values
 bads: 81 items (EMPTY, EMPTY_2, EMPTY_3, EMPTY_4, REF1, REF2, ...)
 ch_names: EMPTY, EMPTY_2, EMPTY_3, EMPTY_4, REF1, REF2, AL1, AL2, AL3, ...
 chs: 272 sEEG, 1 Stimulus
 custom_ref_applied: False
 dig: 0 items
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 273
 projs: []
 sfreq: 2000.0 Hz
>

## Bipolar Rereferencing

In [5]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
    matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
    if matcher:
        probe, num = matcher.groups()
        return probe, int(num)
    else:
        return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [6]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            # print(ch, pair)

EMPTY noref
EMPTY_2 noref
EMPTY_3 noref
EMPTY_4 noref
REF1 noref
REF2 noref
BL1 noref
BL16 noref
DL14 noref
EL1 noref
EL2 noref
EL12 noref
FL12 noref
GL1 noref
GL2 noref
GL3 noref
GL15 noref
GL16 noref
HL1 noref
HL2 noref
HL3 noref
HL4 noref
HL14 noref
HL15 noref
HL16 noref
IL1 noref
IL2 noref
IL3 noref
IL16 noref
JL14 noref
KL16 noref
NR12 noref
OR16 noref
EMPTY_227 noref
EMPTY_228 noref
FP1 noref
F3 noref
C3 noref
P3 noref
O1 noref
FP2 noref
F4 noref
C4 noref
P4 noref
O2 noref
F7 noref
T7 noref
P7 noref
F8 noref
T8 noref
P8 noref
F9 noref
F10 noref
FPZ noref
FZ noref
CZ noref
PZ noref
OZ noref
Ekg1 noref
ekg2 noref
EMPTY_254 noref
EMPTY_255 noref
EMPTY_256 noref
DC01 noref
DC02 noref
DC03 noref
DC04 noref
DC05 noref
DC06 noref
DC07 noref
DC08 noref
DC09 noref
DC10 noref
DC11 noref
DC12 noref
DC13 noref
DC14 noref
DC15 noref
DC16 noref
STI noref


In [7]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 5901899  =      0.000 ...  2950.950 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=178, n_times=5901900
    Range : 0 ... 5901899 =      0.000 ...  2950.950 secs
Ready.
Added the following bipolar channels:
AL1-AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, AL12-AL13, AL13-AL14, AL14-AL15, AL15-AL16, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL6, BL6-BL7, BL7-BL8, BL8-BL9, BL9-BL10, BL10-BL11, BL11-BL12, BL12-BL13, BL13-BL14, BL14-BL15, CL1-CL2, CL2-CL3, CL3-CL4, CL4-CL5, CL5-CL6, CL6-CL7, CL7-CL8, CL8-CL9, CL9-CL10, CL10-CL11, CL11-CL12, CL12-CL13, CL13-CL14, DL1-DL2, DL2-DL3, DL3-DL4, DL4-DL5, DL5-DL6, DL6-DL7, DL7-DL8, DL8-DL9, DL9-DL10, DL10-DL11, DL11-DL12, DL12-DL13, EL3-EL4, EL4-EL5, EL5-EL6, EL6-EL7, EL7-EL8, EL8-EL9, EL9-EL10, EL10-EL11, EL11-EL13, EL13-EL14, FL1-FL2, FL2-FL3, FL3-FL4, FL4-FL5, FL5-FL6, FL6-FL7, FL7-FL8, FL8-FL9, FL9-FL10, FL10-FL11, GL4-GL5, GL5-GL6,

/tmp/ipykernel_2186/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_data-1.fif
Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_data-2.fif
Closing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_data-2.fif
Closing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_data.fif
[done]
Reading 0 ... 5901899  =      0.000 ...  2950.950 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=178, n_times=5901900
    Range : 0 ... 5901899 =      0.000 ...  2950.950 secs
Ready.
Added the following bipolar channels:
AL1-AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, AL12-AL13, AL13-AL14, AL14-AL15, AL15-AL16, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL6, BL6-BL7, BL7-BL8, BL8-BL9, BL9-

/tmp/ipykernel_2186/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_raw_clean_data-1.fif
Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_raw_clean_data-2.fif
Closing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_raw_clean_data-2.fif
Closing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_raw_clean_data.fif
[done]


In [ ]:
# Visualize it #

filtered_notch_bp_data.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

## Epoching the data 

Alright, the longest remaining trial is 14 seconds... so we will epoch with 1 seconds before and 14s afterwards. But because we need padding for filtering the signal we need to add .75s (1500 samples) on either side.

### Onset

In [ ]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 17, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

### Trial End

In [ ]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

### Last Away

In [ ]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

In [ ]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [ ]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


### First Dot

In [ ]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

In [ ]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [ ]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


## First Move

In [ ]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

In [ ]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [ ]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')
